![MLU Logo](../images/MLU_Logo.png)

# Ansel Blume MLU-NLP2 Final Project

## Problem Statement
The project focuses on answer selection and uses the WikiQA dataset. Each record in the dataset has a question, answer and relevance score. The relevance score is binary, 1/0 indicating whether the answer is relevant to the question. 

Each question can be repeated multiple times and can have multiple relevant answer statements. 

To make the problem less complex, we have considered only questions which have at least 1 relevant answer. This simplification results in train, validation and test datasets with 873, 126 and 243 questions respectively.

## Project Objective

In this notebook, you will start our jorney. It contains a baseline model that will give you a first performance score and ourse and all code necessary ready for your first submission.

__IMPORTANT__ 

Make sure you submit this notebook to get to know better how Leaderboard works and, also, make sure your completion will be granted :) .

## The Baseline Model

Here we are using Torchtext: an NLP specific package in Torch. 

We will generate 100 dim vector embeddings for each word using Glove and build a basic convolutional network which takes the text embeddings as input (50 * 100). The training dataset is trained in batches using this network and the losses in each epoch are backpropagated to update the weights and minimize losses in future iterations.

The trained model is then used to make predictions on test dataset and finally, a result dataset with the list of predictions and sequential ID is created for your first leaderboard submission

Notebook has been inspired from https://www.kaggle.com/ziliwang/pytorch-text-cnn

### __Dataset:__
The originial train and test datasets have questions for which there are no answers with relevance 1. To make the problem simpler, we have considered only questions which have atleast 1 answer with relevance score 1. This updated version of the datasets are used in the project

### __Table of Contents__
Here is the plan for this assignment.
<p>
<div class="lev1">
    <a href="#Reading the dataset"><span class="toc-item-num">1&nbsp;&nbsp;</span>
        Reading the dataset
    </a>
</div>
<div class="lev1">
    <a href="#Data-Preparation"><span class="toc-item-num">2&nbsp;&nbsp;</span>
        Data Preparation
    </a>
</div>
<div class="lev1">
    <a href="#Model-Building"><span class="toc-item-num">3&nbsp;&nbsp;</span>
        Model Building
    </a>
</div>
<div class="lev1">
    <a href="#Training"><span class="toc-item-num">4&nbsp;&nbsp;</span>
        Training
    </a>
</div>
<div class="lev1">
    <a href="#Prediction"><span class="toc-item-num">5&nbsp;&nbsp;</span>
        Prediction
    </a>
</div>
<div class="lev1">
    <a href="#Submit-Results"><span class="toc-item-num">6&nbsp;&nbsp;</span>
        Submit Results
    </a>
</div>

In [1]:
import pandas as pd
import os
import numpy as np
import torch
from torch import nn
from sklearn.metrics import f1_score

### Reading the dataset
The datasets are in our MLU datalake and can be downloaded to your local instance here

In [2]:
TRAIN_DATA_FILE ='./training.csv'
TEST_DATA_FILE = './public_test_features.csv'

Below, we are combining question and answer in each row as 1 single text column for simplicity. Alternatively, we can run two parallel networks for question and answer, merge the output of the 2 networks and have a classification layer as output. You may choose to save the files for ease of use, in future steps.

In [3]:
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import RobertaTokenizer
from pprint import pprint

tokenizer = RobertaTokenizer.from_pretrained('roberta-large-mnli')

train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

class RelevanceDataset(Dataset):        
    def __init__(self, df, use_weights=False):
        self.use_weights = use_weights
        self.df = df.copy()
        self.df.rename(columns={'relevance': 'labels'}, inplace=True)
        
        # Encodes <s>question</s></s>answer</s> pairwise
        tokenized = tokenizer(
            list(self.df['question']), 
            list(self.df['answer']),
            return_tensors='pt',
            padding='max_length'
        )
        
        # Need to wrap in list as otherwise pd gets confused and breaks up tensor
        self.df['input_ids'] = list(tokenized['input_ids'])
        self.df['attention_mask'] = list(tokenized['attention_mask'])
                
        # Build weights to increase recall
        if 'labels' in self.df:      
            self.weights = self._compute_weights() if use_weights else [1, 1]
            self.df['weights'] = self.df['labels'].apply(lambda x: self.weights[x])
        
    def __getitem__(self, i):
        return dict(self.df.iloc[i])
    
    def __len__(self):
        return len(self.df)
    
    def _compute_weights(self):
        value_counts = self.df['labels'].value_counts()
        return value_counts.max() / value_counts
    
def get_datasets(args):
    train_dataset = RelevanceDataset(train, use_weights=args.use_weights)
    test_set = RelevanceDataset(test)

    train_frac = args.train_frac if 'train_frac' in args else .9
    train_set_len = int(len(train_dataset) * train_frac)
    val_set_len = len(train_dataset) - train_set_len
    train_set, val_set = random_split(
        train_dataset, 
        [train_set_len, val_set_len],
        generator=torch.Generator().manual_seed(42)
    )
    #train_set, val_set, _ = random_split(train_dataset, [100, 30, len(train_dataset) - 130])
    
    return train_set, val_set, test_set, train_dataset

### Data Preparation

In [4]:
import torch
from collections import defaultdict

def collate_fn(dicts):
    batch_dict = defaultdict(list)
    
    for d in dicts:
        for k, v in d.items():
            batch_dict[k].append(v)
                        
    for k in ['input_ids', 'attention_mask']:
        batch_dict[k] = torch.stack(batch_dict[k])
        
    if 'labels' in batch_dict:
        for k in ['labels', 'weights']:
            batch_dict[k] = torch.tensor(batch_dict[k])
    
    return dict(batch_dict)

def batch_to_device(batch, device):
    for k, v in batch.items():
        if type(v) == torch.Tensor:
            batch[k] = v.to(device)
            
    return batch

### Model Building

In [5]:
from transformers import RobertaModel
import torch.nn as nn

class RobertaRelevanceClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = RobertaModel.from_pretrained('roberta-large-mnli')
        self.embed_dim = self.model.embeddings.word_embeddings.weight.shape[-1]
        
        self.classifier = nn.Sequential(
            nn.Linear(self.embed_dim, self.embed_dim),
            nn.ReLU(),
            nn.Linear(self.embed_dim, 1)
        )
        
    def forward(self, input_ids, attention_mask):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(output.pooler_output)
        
        return logits

### Training

Below is the training loop. Each batch of training data is read, predictions are computed through forward propagation of batch inputs. Losses are computed between predictions and actual labels and back propagated to update the weights. In each epoch, we compute the f1 score with a preset threshold of 0.15 (this can be a tunable parameter and could provide better performance of other thresholds)

In [6]:
from transformers import get_constant_schedule_with_warmup
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast
import torchmetrics as tm
import json
import re

class TrainArgs:
    def __init__(self, **kwargs):
        self.hparams = kwargs
        self.__dict__.update(**self.hparams)
        
    def __contains__(self, arg):
        return arg in self.hparams
    
    def __getitem__(self, arg):
        return self.hparams[arg]
    
    def __setitem__(self, arg, val):
        self.hparams[arg] = val
        setattr(self, arg, val)

def run_training(args, model, train_ds, val_ds, writer):
    # Helper function to compute logits and loss
    def logits_and_loss(batch, model, args):
        batch_to_device(batch, args.device)
        with autocast():
            logits = model(batch['input_ids'], batch['attention_mask']).flatten()
            loss = F.binary_cross_entropy_with_logits(
                logits, 
                batch['labels'].float(),
                weight=batch['weights'] if args.use_weights else None
            )
            loss = loss / args.accum_grad_batches
        
        return logits.detach(), loss
    
    # Helper function to take an optimizer step
    def optimizer_step(optimizer, scheduler, scaler):
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        optimizer.zero_grad()
    
    # Metrics
    pred_threshold = args.pred_threshold if 'pred_threshold' in args else .5
    train_metrics = tm.MetricCollection([
        tm.Accuracy(threshold=pred_threshold),
        tm.Precision(threshold=pred_threshold),
        tm.Recall(threshold=pred_threshold),
        tm.F1(threshold=pred_threshold),
        tm.AveragePrecision()
    ]).to(args.device)
    val_metrics = train_metrics.clone().to(args.device)
        
    # Set up dataloaders
    dl_kwargs = {
        'batch_size': args.batch_size,
        'collate_fn': collate_fn,
        'shuffle': True
    }
    train_dl = DataLoader(train_ds, **dl_kwargs)
    val_dl = DataLoader(val_ds, **dl_kwargs)
    
    # Training loop
    model.to(args.device)
    
    optimizer = torch.optim.AdamW(
        model.parameters(), 
        lr=args.lr,
        weight_decay=args.weight_decay if 'weight_decay' in args else .01
    )
    scheduler = get_constant_schedule_with_warmup(optimizer, args.warmup_steps)
    scaler = GradScaler()
    
    starting_epoch = 0 if 'starting_epoch' not in args else args.starting_epoch    
    for epoch in range(starting_epoch, starting_epoch + args.epochs):
        print(f'Epoch {epoch}')
        
        # Train
        model.train()
        losses = []
        print('Training')
        for i, batch in enumerate(tqdm(train_dl)):
            logits, loss = logits_and_loss(batch, model, args)
            
            # Metrics
            losses.append(loss.item())
            predictions = logits.sigmoid()
            train_metrics(predictions, batch['labels'])
            
            # Optimization
            scaler.scale(loss).backward()
            
            if (i + 1) % args.accum_grad_batches == 0 or i == len(train_dl) - 1:
                tqdm.write('Taking a step')
                optimizer_step(optimizer, scheduler, scaler)
            
                # Print metrics for the accumulated batch
                tqdm.write(f'Predictions: {predictions}')
                tqdm.write(f'Labels: {batch["labels"]}')
                tqdm.write(str(train_metrics.compute()))
            
        writer.add_scalar('Loss/train', torch.tensor(losses).mean(), epoch)
        writer.add_scalars('Train', train_metrics.compute(), epoch)
        
        # Validate
        model.eval()
        losses = []
        print('Validating')
        for batch in tqdm(val_dl):
            with torch.no_grad():
                logits, loss = logits_and_loss(batch, model, args)
                
            losses.append(loss.item())
            predictions = logits.sigmoid()
            val_metrics(predictions, batch['labels'])
        
        writer.add_scalar('Loss/val', torch.tensor(losses).mean(), epoch)
        writer.add_scalars('Val', val_metrics.compute(), epoch)
        tqdm.write(str(val_metrics.compute()))

        # Save model
        torch.save(model.state_dict(), f'checkpoints/epoch={epoch}_state_dict.pkl')
        
        # Reset metrics
        train_metrics.reset()
        val_metrics.reset()
        
    writer.flush()
    
def load_model(args):
    checkpoint_re = r'.+=(\d+).+'
    model = RobertaRelevanceClassifier()
    
    if 'checkpoint' in args:
        saved_epoch = int(re.match(checkpoint_re, args.checkpoint).group(1))
        args['starting_epoch'] = saved_epoch + 1
        model.load_state_dict(torch.load(args.checkpoint))
        
    if 'freeze_roberta' in args:
        for parameter in model.model.parameters():
            parameter.requires_grad = not args.freeze_roberta
    
    return model

In [ ]:
args = TrainArgs(
    batch_size=2,
    epochs=10,
    device='cuda:0' if torch.cuda.is_available() else 'cpu',
    lr=1e-5,
    use_weights=True,
    accum_grad_batches=256,
    warmup_steps=10,
    freeze_roberta=False,
    pred_threshold=.5,
    train_frac=.9,
    weight_decay=.01,
    checkpoint='checkpoints/epoch=10_state_dict.pkl'
)

# Tensorboard logging
# https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html
# https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks
writer = SummaryWriter()

model = load_model(args)
train_set, val_set, test_set, train_dataset = get_datasets(args)
run_training(args, model, train_set, val_set, writer)

Setting random seed for reproducibility. Batch size is set as 512 and is a hyperparameter that can be varied. Torch uses bucket iterator for language modelling tasks. 

Below steps are used to define the model, initialize the model, the optimization algorithm and loss function. Learning rate is a tunable hyperparameter here

%%time
training(3, model, loss_function, optimizer, train_iter)

### Prediction

Below function is used to predict on test dataset using trained model. It returns a list of predicted probabilities

In [16]:
def predict(args, model, test_ds):
    # Model to correct device
    model.to(args.device)
    model.eval()
    
    # Set up data and metrics
    dl = DataLoader(test_ds, batch_size=args.batch_size)
    pred_l = []
    
    metrics = tm.MetricCollection([
        tm.Accuracy(threshold=args.pred_threshold),
        tm.Precision(threshold=args.pred_threshold),
        tm.Recall(threshold=args.pred_threshold),
        tm.F1(threshold=args.pred_threshold),
        tm.AveragePrecision()
    ]).to(args.device)
    
    has_labels = 'labels' in next(iter(dl))
    
    # Predict
    for batch in tqdm(dl):
        batch_to_device(batch, args.device)
        
        with torch.no_grad():
            probs = model(batch['input_ids'], batch['attention_mask']).flatten().sigmoid()
            
        preds = (probs > args.pred_threshold).int()
        pred_l.append(preds.cpu())
            
        if has_labels:
            metrics(probs, batch['labels'])
    
    return torch.cat(pred_l).cpu(), metrics.cpu().compute() if has_labels else None

In [17]:
args = TrainArgs(
    checkpoint='checkpoints/epoch=10_state_dict.pkl',
    pred_threshold=.5,
    use_weights=False,
    device='cuda:0' if torch.cuda.is_available() else 'cpu',
    batch_size=500
)
train_set, val_set, test_set, train_dataset = get_datasets(args)
model = load_model(args)
preds, metrics = predict(args, model, test_set)

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaModel: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 6/6 [01:28<00:00, 14.78s/it]


In [ ]:
metrics

### Submit Results

Create a new dataframe for submission. The list of predicted probabilities are converted to labels using the pre-defined threshold of 0.15 (can be tuned for better performance). The list of labels is concatenated with the original sequential ID from the test file downloaded from Leaderboard, to generate the final submission

For submission, follow these steps:
1. Go to the folder where your notebook is in Sagemaker
2. Donwload the file __test_submission_nlp2.csv__ to your local machine
3. On NLP2 Leaderboard contest, select option __My Submissions"__ and upload your file

In [86]:
result_df = pd.DataFrame(columns=["ID", "relevance"])
result_df["ID"] = test_set.df["ID"].tolist()
result_df["relevance"] = preds
result_df.to_csv("test_submission_nlp2.csv", index=False)

## Random forest on embeddings

In [71]:
def get_embeds_and_labels(args, model, ds):
    embeddings = []
    labels = []
    dl = DataLoader(ds, batch_size=args.batch_size)
    for batch in tqdm(dl):
        batch_to_device(batch, args.device)
        with torch.no_grad():
            batch_embedding = model.model(
                batch['input_ids'], 
                attention_mask=batch['attention_mask']
            ).pooler_output
        embeddings.append(batch_embedding.cpu())
        
        if 'labels' in batch:
            labels.append(batch['labels'].cpu())
        
    return torch.cat(embeddings), torch.cat(labels) if len(labels) > 0 else None

In [30]:
train_embeds, train_labels = get_embeds_and_labels(args, model, train_set)

torch.Size([6174, 1024])

In [42]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(train_embeddings, train_labels)

RandomForestClassifier()

In [49]:
val_embeds, val_labels = get_embeds_and_labels(args, model, val_set) 

100%|██████████| 2/2 [00:20<00:00, 10.34s/it]


In [88]:
from sklearn.metrics import precision_recall_fscore_support

outputs = clf.predict(val_embeds)
results = precision_recall_fscore_support(val_labels, outputs, average='micro')

In [72]:
test_embeds, None = get_embeds_and_labels(args, model, test_set)

100%|██████████| 6/6 [01:28<00:00, 14.70s/it]


In [84]:
preds = clf.predict(test_embeds)